In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Leer Dataframes
books = pd.read_excel('books_data.xlsx')
ratings = pd.read_excel('Books_rating.xlsx')

In [3]:
print(len(books))   #número de filas del dataframe books
print(len(ratings)) #número de filas del dataframe ratings

212404
1048575


In [4]:
books = books[['Title','description','categories']]
ratings = ratings[['Title','User_id','review/score']]
books = books.dropna().reset_index()
ratings = ratings.dropna().reset_index()

In [5]:
books=books.sample(n=30000, random_state=42).reset_index() #solo tomamos 100000 filas del dataframe books porque el codigo no carga mas adelante con mas datos

In [6]:
ratings=ratings[ratings['Title'].isin(books['Title'])] # filtramos el dataframe con los ratings solo a los libros de los que tenemos calificaciones

In [7]:
ratings=ratings.sample(n=30000, random_state=42).reset_index()

In [8]:
print(len(books))   #número de filas del dataframe books para corroborar la cantidad de filas
print(len(ratings)) #número de filas del dataframe ratings para corroborar la cantidad de filas

30000
30000


In [9]:
#Analizamos los dataframes
print(books.columns)
print(books.shape)
print(ratings.columns)
print(ratings.shape)
print(books.info())
print(ratings.info())

Index(['level_0', 'index', 'Title', 'description', 'categories'], dtype='object')
(30000, 5)
Index(['level_0', 'index', 'Title', 'User_id', 'review/score'], dtype='object')
(30000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   level_0      30000 non-null  int64 
 1   index        30000 non-null  int64 
 2   Title        30000 non-null  object
 3   description  30000 non-null  object
 4   categories   30000 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   level_0       30000 non-null  int64 
 1   index         30000 non-null  int64 
 2   Title         30000 non-null  object
 3   User_id       30000 non-null  object
 4   r

In [10]:
print(books.head())
print(ratings.head())

   level_0   index                                              Title  \
0    25417   35577                                    Rampaging Bulls   
1    92790  140829            The Principles of Statistical Mechanics   
2    35349   50009  Financial Engineering and Computation: Princip...   
3    25845   36168  Adobe Scripting: Your Visual Blueprint to Scri...   
4   123605  187497  Diagnosing and Treating Codependence: A Guide ...   

                                         description        categories  
0  Rampaging Bulls, written by an investor and wr...  ['Penny stocks']  
1  This is the definitive treatise on the fundame...       ['Science']  
2  Students and professionals intending to work i...   ['Mathematics']  
3  * Written by New York graphic design professio...     ['Computers']  
4  In Practicing Prodependence: The Clinical Alte...    ['Psychology']  
   level_0   index                                              Title  \
0   695974  860439   A Drummer's Beat to Mend (Lov

### Primero realizamos un sistema de recomendación basado en contenido para esto utilizaremos el dataframe "Books" ya que necesitamos las características de los libros, el cual en este dataframe esta informacion se encuentra en la descripción, para calcular la similitud entre ellos y recomendar elementos similares

Las columnas de nuestro interés para hacer las recomendaciones son 'description' y 'categories' asi que las combinaremos

In [11]:
#Los datos de la columna categories tienen el siguiente formato: '['Comics & Graphic Novels']' por lo que eliminaremos los simbolos [' y ']
#Por otro lado de la columna 'description' quitaremos los puntos y comas todo esto con el fin de que el código pueda relacionar el conteido correctamente y no descarte similitudes por un punto entre palabras 

books['categories'] = books['categories'].str.replace("\']", '').str.replace("\['", '')
books['description'] = books['description'].str.replace("\.", '').str.replace("\,", '')
#Observamos como quedan las columna pidiendole UN valor de cada una como ejemplo
print(books['categories'].sample(1))
print(books['description'].sample(1))

18040    ['Fiction
Name: categories, dtype: object
13152    Effectively integrating grammar instruction wi...
Name: description, dtype: object


In [12]:
# Combinamos columnas 'description' y 'categories' para nuestras recomendaciones
books['combined'] = books['description'] + ' ' + books['categories']
print(books['combined'].sample(1))

10145    Often viewed as nostalgic and inauthentic, the...
Name: combined, dtype: object


In [13]:
#Ahora vamos a vamos a representar los items en un formato que la máquina pueda entender, Con la vectorización. Para convertir texto en una representación numérica 
count = TfidfVectorizer()
count_matrix = count.fit_transform(books['combined'])

In [14]:
# Calcular la similitud entre las descripciones de los libros
# Usaremos la similitud del coseno
similarity = cosine_similarity(count_matrix,count_matrix)

In [17]:
#Ahora crearemos una fucnión que a agregar el título de un libro devuelva los libros mas similares a el 
def recommendations(Title, similarity):
    recommended_books = []

    # Verificar si el título está presente en el DataFrame
    if Title in books['Title'].values:
        # Obtenemos el índice del libro que coincide con el elegido
        indice = books[books['Title'] == Title].index[0]

        # Obtenemos los scores de similitud de coseno para ese libro con todos los libros
        score_series = pd.Series(similarity[indice]).sort_values(ascending=False)

        # Obtenemos los 10 indices de libros con mayor similitud
        top_10 = list(score_series.iloc[1:11].index)

        for i in top_10:
            recommended_books.append(books['Title'][i])
    else:
        print("El título del libro no se encuentra en los datos.")

    return recommended_books

In [18]:
book_title = 'Banishment'
recommended__books = recommendations(book_title, similarity=similarity)

print(f"Libros recomendados para '{book_title}':")
for book in recommended__books:
    print(book)

Libros recomendados para 'Banishment':
Rebels with a Cause: The Minds and Morality of Political Offenders
Historic Towns of the Western States
Historic towns of the western states
Pueblo, Hardscrabble, Greenhorn: Society on the High Plains, 1832-1856
Kinkeeper (New Women's Voices Series, No. 18)
The Maritime History of the World **Volumes 1 & 2**
Colonial Odysseys: Empire and Epic in the Modernist Novel
The History of Government from the Earliest Times: Ancient Monarchies and Empires; The Intermediate Ages; Empires, Monarchies and the Modern State (3 Volume Set)
The Italian Boy: A Tale of Murder and Body Snatching in 1830s London
Gentle Invaders: Quaker Women Educators and Racial Issues During the Civil War and Reconstruction


### Ahora realizamos un sistema de recomendación de filtro colaborativo para esto utilizaremos el dataframe "Ratings" ya que necesitamos información sobre las interacciones entre usuarios y libros, como las calificaciones o las compras realizadas.

Las columnas de nuestro interés para hacer las recomendaciones son 'description' y 'categories' asi que las combinaremos

In [130]:
# Creamos una matriz de puntaje, para eso utilizamos los Id de usuario como index y el Titulo de libro como columnas y el contenido las reviews
ratings_matrix = ratings.pivot_table(index='User_id',columns='Title', values='review/score',aggfunc='mean').fillna(0)

In [48]:
# Calculamos la similitud entre usuarios basada en las calificaciones utilizando la similitud del coseno
User_similarity = cosine_similarity(ratings_matrix.fillna(0))

In [135]:
def collaborative_filtering(User_id, ratings_matrix, User_similarity, top_n=10):
    # Obtener el índice numérico correspondiente al ID del usuario
    User_index = ratings_matrix.index.get_loc(User_id)
    
    # Obtener las similitudes del usuario dado con todos los usuarios
    User_similarities = User_similarity[User_index]
    
    # Obtener los índices de los usuarios más similares al usuario dado (excluyendo al usuario dado)
    similar_user_indices = np.argsort(User_similarities)[::-1][1:top_n+1]
    
    # Obtener las calificaciones del usuario dado
    User_ratings = ratings_matrix.iloc[User_index].fillna(0)
    
    # Obtener las calificaciones promedio de los usuarios similares
    similar_users_ratings = ratings_matrix.iloc[similar_user_indices].mean()
    
    # Obtener los libros que los usuarios similares han calificado positivamente y que el usuario dado no ha calificado
    recommended_books = similar_users_ratings[User_ratings == 0].sort_values(ascending=False)[:top_n]
    
    return recommended_books.index

In [136]:
User_id = 'A1CNQTCRQ35IMM'
recommended_books = collaborative_filtering(User_id, ratings_matrix, User_similarity=User_similarity)
print(f"Recomendaciones para el usuario {User_id}:")
for Title in recommended_books:
    print(Title)

Recomendaciones para el usuario A1CNQTCRQ35IMM:
"THE GOBLET AND THE RESTLESS GHOST"
Starburst
Statistical Inference: An Integrated Approach (Chapman & Hall/CRC Texts in Statistical Science Series)
Statistical Analysis in Climate Research
Statistical Analyses for Language Assessment Workbook and CD ROM (Cambridge Language Assessment)
Stash Envy
Start Your Own Specialty Travel & Tour Business (Start Your Own Specialty Travel & Tour Business)
Start Making Sense: Turning the Lessons of Election 2004 into Winning Progressive Politics
Stars (Gem)
Starlight


## Evaluar los sistemas


In [163]:
#Definimos la funcion para calcular la precisión k para el modelo basado en contenido
def precision_k(recommended_books, test_ratings, k):
    # Tomar los primeros k elementos de los libros recomendados
    recommended_k = recommended_books[:k]
    
    # Calcular la intersección entre los libros recomendados y los libros reales
    intersection = set(recommended_k) & set(lista_real)
    
    # Calcular la precisión en k
    precision = len(intersection) / k
    
    return precision

#Definimos la funcion para calcular la precisión k para el modelo de filtro colaborativo

def precision_at_k(User_id, recommended_books, test_ratings, k):
    # Filtrar las calificaciones de prueba para el usuario dado
    test_user_ratings = test_ratings[test_ratings['User_id'] == User_id]
    
    # Obtener los libros verdaderamente relevantes (calificados positivamente) por el usuario en los datos de prueba
    relevant_books = test_user_ratings[test_user_ratings['Title'].isin(recommended_books)]
    
    # Calcular la precisión en k
    precision = len(relevant_books) / k
    
    return precision


In [164]:
books['Title'] = books['Title'].str.replace("\'", '') #Elimina caracteres que pueden molestar en el código 


In [166]:
print(ratings['User_id'].sample(1)) #para sacar un ejemplo de usuario real

20875    A3RAT5GIH32UNM
Name: User_id, dtype: object


In [189]:

User_id = 'A1CNQTCRQ35IMM'
test_ratings = lista_real(User_id)
print(test_ratings)


['Saving Grace', 'Little Women', 'Carrie', 'The Girl Who Loved Tom Gordon', 'Dead Cert', 'Running Blind', 'Visitor', 'Hot Shot']


In [190]:
book_title = test_ratings[0] #toma el primer libro de la lista para sacar las recomendaciones
print(book_title)

Saving Grace


In [191]:
#libros recomendados para el modelo basado en contenido
recommended_books_content_based = recommendations(book_title, similarity=similarity)
print(f"Libros recomendados para '{book_title}':")
for book in recommended_books_content_based:
    print(book)

Libros recomendados para 'Saving Grace':
Society: The Basics (6th Edition)
The Book of Understanding: Creating Your Own Path to Freedom
Grace and Truth
Unnatural Affection
The Beauty of the Beast: Breathing New Life Into Organizations
Beyond the Twelve Steps: Roadmap to a New Life
Alias Grace: Complete & Unabridged
Love for an Enemy
Interview With An American Monk
Human/Nature


In [188]:
#libros recomendados para el modelo basado en filtro colaborativo
recommended_books_collaborative_filtering = collaborative_filtering(User_id, ratings_matrix, User_similarity=User_similarity)
print(f"Recomendaciones para el usuario {User_id}:")
for Title in recommended_books_collaborative_filtering:
    print(Title)

Recomendaciones para el usuario A1CNQTCRQ35IMM:
"THE GOBLET AND THE RESTLESS GHOST"
Starburst
Statistical Inference: An Integrated Approach (Chapman & Hall/CRC Texts in Statistical Science Series)
Statistical Analysis in Climate Research
Statistical Analyses for Language Assessment Workbook and CD ROM (Cambridge Language Assessment)
Stash Envy
Start Your Own Specialty Travel & Tour Business (Start Your Own Specialty Travel & Tour Business)
Start Making Sense: Turning the Lessons of Election 2004 into Winning Progressive Politics
Stars (Gem)
Starlight


In [197]:
#calculamos la presicion K para el modelo basado en contenido
precision_content_based = precision_k(recommended_books_content_based, test_ratings)
print(precision_content_based)

0.0


In [202]:
#calculamos la presicion K para el modelo basado en filtro colaborativo
k=10
precision_collaborative_filtering = precision_at_k(User_id, recommended_books_collaborative_filtering, ratings, k)
print(precision_collaborative_filtering)

0.0
